# 110. Basic Linear Regression

In [1]:
import numpy as np
import torch

<h1>1) torch.nn 의 Linear 클래스를 이용한 Linear Regression</h1>

In [2]:
from torch.nn import Linear
torch.manual_seed(1)

model = Linear(in_features=1, out_features=1, bias=True)

<code>state_dict()</code> method 는 각 layer 의 parameter tensor 를 dictionary 로 반환하므로 직접 값을 update 할 수 있다.

In [3]:
print(model.state_dict())
print()
print(model.state_dict()['weight'], model.state_dict()['bias'])
print()
print(model.weight)
print(model.bias)

OrderedDict([('weight', tensor([[0.5153]])), ('bias', tensor([-0.4414]))])

tensor([[0.5153]]) tensor([-0.4414])

Parameter containing:
tensor([[0.5153]], requires_grad=True)
Parameter containing:
tensor([-0.4414], requires_grad=True)


작동 원리 이해를 위해 weight와 bias 값을 임의로 assign

In [4]:
model.state_dict()['weight'][0] = torch.tensor(2)
model.state_dict()['bias'][0]     = torch.tensor(-1)

In [5]:
model.state_dict()

OrderedDict([('weight', tensor([[2.]])), ('bias', tensor([-1.]))])

Neural Network을 이용하여 $y = wx + b$ 계산

In [6]:
# x = [[3.0]]일 경우 prediction = 2*3 - 1 = 5
x = torch.tensor([[3.0]])
yhat = model(x)
yhat.item()

5.0

In [7]:
# x 가 multiple inputs (array) 인 경우:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
print(yhat.detach().numpy())

[[1.]
 [3.]
 [5.]]


<h1>2) 사용자 정의 Neural Network 작성</h1>

In [8]:
from torch import nn

class LR(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [9]:
model = LR(1, 1)

model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.1939]])),
             ('linear.bias', tensor([0.4694]))])

- 위에서와 마찬가지로 임의로 weight와 bias를 assign  
- `model.parameters()` 함수는 PyTorch 모델의 모든 가중치와 편향(bias)을 포함하는 파라미터를 반환  
    - requires_grad=True 는 해당 텐서에 대해 자동 미분을 수행하도록 설정합니다. 즉, 텐서에 대한 연산이 추적되며, 그래디언트 계산을 위한 연산 그래프가 구성됩니다. 이는 학습 가능한 모델 파라미터(가중치와 편향 등)에서 주로 사용됩니다.

In [10]:
model.state_dict()['linear.weight'][0] = torch.tensor(2)
model.state_dict()['linear.bias'][0]     = torch.tensor(-1)

print(list(model.parameters()))
print()
print(model.linear)

[Parameter containing:
tensor([[2.]], requires_grad=True), Parameter containing:
tensor([-1.], requires_grad=True)]

Linear(in_features=1, out_features=1, bias=True)


In [11]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
yhat

tensor([[1.],
        [3.],
        [5.]], grad_fn=<AddmmBackward0>)

In [12]:
yhat.detach().numpy()

array([[1.],
       [3.],
       [5.]], dtype=float32)

# Backpropagation 과 경사하강법 적용

- weight 와 bias를 스스로 학습
- (온도, 강수량, 습도)를 feature 로 입력 받아  (사과, 오렌지)의 수확량(label)을 선형회귀로 예측하는 model 작성

## Data

In [13]:
# Input (습도, 강수량, 최고온도, 최저온도)
inputs = np.array([
                   [73, 67, 43, 10], 
                   [91, 88, 64, 5], 
                   [87, 134, 58, 2], 
                   [102, 43, 37, 4], 
                   [69, 96, 70, 5]], dtype='float32')

In [14]:
# Targets - (apples, oranges) 수확량
targets = np.array([
                    [56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

- input 과 target을 tensor로 변환

In [15]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs.size())
print(targets.size())

torch.Size([5, 4])
torch.Size([5, 2])


# 1) torch.nn Linear Class 사용

In [16]:
from torch.nn import Linear
from torch import optim

torch.manual_seed(1)

In [17]:
model = Linear(in_features=4, out_features=2)

In [18]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [19]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [20]:
preds = model(inputs)
loss = criterion(preds, targets)
print(loss)

tensor(4.0758, grad_fn=<MseLossBackward0>)


In [21]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 58.021572  69.74707 ]
 [ 83.15649   98.7855  ]
 [119.965004 128.4409  ]
 [ 21.83963   34.988026]
 [102.8034   117.893616]]


# 2) 사용자 정의 model 사용

- Linear Regression 사용자 정의 model 

In [22]:
class LinearReg(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        yhat = self.fc(x)
        return yhat

In [23]:
model = LinearReg(inputs.shape[1], targets.shape[1])

In [24]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [25]:
print(model)
print(model.state_dict())

LinearReg(
  (fc): Linear(in_features=4, out_features=2, bias=True)
)
OrderedDict([('fc.weight', tensor([[ 0.1387,  0.0247,  0.1826, -0.1949],
        [-0.0365, -0.0450,  0.0725, -0.0020]])), ('fc.bias', tensor([0.4371, 0.1556]))])


In [26]:
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0001,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'foreach': None,
   'differentiable': False,
   'params': [0, 1]}]}

## Train the model

In [27]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

model.parameters(): 이 함수는 모델의 모든 파라미터를 생성자(generator) 형태로 반환합니다. 이는 주로 모델의 파라미터를 순회(iterate)하거나, 파라미터에 대한 그래디언트를 0으로 초기화하거나, 역전파를 위해 그래디언트를 사용하는 등의 작업에 사용됩니다.

model.state_dict(): 이 함수는 모델의 상태를 Python 딕셔너리 형태로 반환합니다. 이 딕셔너리는 각 계층의 파라미터를 표현하는 텐서를 포함하며, 모델의 전체 상태를 표현합니다. 이 state_dict는 모델의 가중치를 저장하고 불러오는 데 사용되며, 모델의 학습 상태를 체크포인트로 저장하거나, 학습을 다시 시작할 때 이전 상태를 불러오는 등의 작업에 주로 사용됩니다.

간단히 말해, model.parameters()는 모델의 파라미터를 바로 조작하는 데 사용되며, model.state_dict()는 모델의 상태를 저장하고 불러오는 데 사용

In [28]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
         [-0.3081,  0.7807,  0.9301, -0.0332]], requires_grad=True),
 Parameter containing:
 tensor([0.4331, 0.1568], requires_grad=True)]

In [29]:
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
                      [-0.3081,  0.7807,  0.9301, -0.0332]])),
             ('fc.bias', tensor([0.4331, 0.1568]))])

In [30]:
preds = model(inputs)

In [31]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.842937  69.6347  ]
 [ 85.32951  100.18176 ]
 [125.2741   131.84752 ]
 [ 24.323542  36.581146]
 [104.19539  118.78626 ]]


## Save and Load Model

In [32]:
# model save
torch.save(model.state_dict(), 'model.pt')

model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
                      [-0.3081,  0.7807,  0.9301, -0.0332]])),
             ('fc.bias', tensor([0.4331, 0.1568]))])

- model reload 및 parameter 복원 확인

In [33]:
reloaded_model = LinearReg(inputs.shape[1], targets.shape[1])
reloaded_model.load_state_dict(torch.load('model.pt'))

reloaded_model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.3590,  0.8959,  0.6336, -0.3657],
                      [-0.3081,  0.7807,  0.9301, -0.0332]])),
             ('fc.bias', tensor([0.4331, 0.1568]))])

In [34]:
preds = reloaded_model(inputs)

In [35]:
# Predictions & target 비교
print(targets.detach().numpy())
print()
print(preds.detach().numpy())

[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]

[[ 57.842937  69.6347  ]
 [ 85.32951  100.18176 ]
 [125.2741   131.84752 ]
 [ 24.323542  36.581146]
 [104.19539  118.78626 ]]
